+ Pixelized shear measurements (smoothing -> pixelize/ pixelize -> smoothing)
+ Get the variance map (mock/analytical)

In [1]:
%pylab inline
import os
import gc
import glob
import numpy as np
import astropy.io.fits as pyfits
import maputil as mapUtil
from configparser import ConfigParser
from pixel3D import cartesianGrid3D
from scipy.stats import norm as statnorm

Populating the interactive namespace from numpy and matplotlib


In [11]:
configName  =   'wrkDir/config-nl15.ini'
parser      =   ConfigParser()
parser.read(configName)

configName  =   'wrkDir/config-nl15.ini'
pDict={'dempz': 'demp',\
       'dnnz': 'dnn',\
       'mizuki': 'mizuki'}

def plotHist(fieldname,array,pixels,oname):
    cmap=plt.get_cmap('tab20')

    plt.figure(figsize=(8,6))

    gbin=plt.hist(array,bins=100,density=True,range=(-1.2,1.2),histtype='step',label='galaxy',color=cmap(0))[1]
    gbinGal=(gbin-np.average(array))/np.std(array)
    a=statnorm.pdf(gbinGal)
    plt.plot(gbin,a/np.sum(a)/(gbin[1]-gbin[0]),color=cmap(0),ls='--')

    gbin=plt.hist(pixels,bins=100,density=True,range=(-1.2,1.2),histtype='step',label='pixel',color=cmap(2))[1]

    gbinPix=(gbin-np.average(pixels))/np.std(pixels)
    a=statnorm.pdf(gbinPix)
    plt.plot(gbin,a/np.sum(a)/(gbin[1]-gbin[0]),color=cmap(2),ls='--')
    plt.title('%s' %fieldname,fontsize=20)
    plt.xlabel(r'$\delta g_1$',fontsize=20)
    plt.ylabel(r'$P(\delta g_1)$',fontsize=20)
    plt.legend(fontsize=20)
    plt.yscale('log')
    plt.ylim(1e-4,15)
    plt.grid()
    plt.tight_layout()
    plt.savefig(oname)
    plt.close()
    return

In [12]:
key='dempz'
colname=pDict[key]
for fieldname in mapUtil.field_names:
    inDir  =   'wrkDir/reGaus_catalog/%s_groups/' %fieldname
    outDir =   'wrkDir/reGaus_pixel3D/%s_groups/' %fieldname
    if not os.path.isdir(outDir):
        os.mkdir(outDir)
    fnames=glob.glob(os.path.join(inDir,'src-group-*.fits'))
    for fname in fnames:
        gn=fname.split('-')[-1].split('.')[0]
        # update zbounds
        bname=os.path.join(outDir,'group-%s-%s-10bins.fits' %(gn,key))
        bounds=pyfits.getdata(bname,ext=1)['zbounds']
        parser.set('sourceZ','zbound','[%s]' %(','.join([str(bb) for bb in bounds])))
        gridInfo    =   cartesianGrid3D(parser)
        glab=fieldname+"-%s"%gn
        # poz average
        data=pyfits.getdata(fname)
        mask=(~np.isnan(data['%s_zbest' %colname]))\
            &(data['%s_zbest_std'%colname]<1.0)\
            &(data['%s_zbest'%colname]<3.0)\
            &(data['%s_zbest'%colname]>0.05)
        data=data[mask]
        wsAve=np.average(data['weight'])
        ws=data['weight']/wsAve
        z=data['%s_zbest'%colname]
        x,y=gridInfo.setupTanPlane(data['ra'],data['dec'])
        out1=gridInfo.pixelize_data(x,y,z,data['g1'],ws=ws)
        out2=gridInfo.pixelize_data(x,y,z,data['g2'],ws=ws)
        outname1=os.path.join(outDir,'group-%s-%s-g1Map.fits' %(gn,key))
        outname2=os.path.join(outDir,'group-%s-%s-g2Map.fits' %(gn,key))
        pyfits.writeto(outname1,out1,overwrite=True)
        pyfits.writeto(outname2,out2,overwrite=True)
        outname3=os.path.join(outDir,'group-%s-%s-g1Hist.png' %(gn,key))
        plotHist(glab,data['g1'],out1[out1!=0],outname3)
        del out1,out2,data,bounds,x,y,z,ws
        gc.collect()

In [3]:
    # w*g1,w*g2,w for each pixel
    g1OutTmp=numpy.histogramdd((z,y,x),bins=(gridInfo.zbound,gridInfo.ybound,gridInfo.xbound), weights=v.real*ws)[0]
    g2OutTmp=numpy.histogramdd((z,y,x),bins=(gridInfo.zbound,gridInfo.ybound,gridInfo.xbound), weights=v.imag*ws)[0]
    weightOutTmp=numpy.histogramdd((z,y,x),bins=(gridInfo.zbound,gridInfo.ybound,gridInfo.xbound), weights=ws)[0]
    mask0=weightOutTmp>0. # the mask before smoothing
    assert weightOutTmp.shape==gridInfo.shape,'the output shape is different from expectation?'
    # variance
    sigma2=np.zeros(gridInfo.shape)
    sigma2[mask0]=1./weightOutTmp[mask0]
    if gridInfo.sigma>0:
        # Gaussian Kernel in Fourier space (normalized in configuration space)
        gausKer=haloSim.GausAtom(ny=ny,nx=nx,sigma=gridInfo.sigma/gridInfo.delta,fou=True,lnorm=2.)
        norm=gausKer[0,0]
        gausKer/=norm
        
        # Gaus**2. in Fourier space
        gausKer2=haloSim.GausAtom(ny=ny,nx=nx,sigma=gridInfo.sigma/gridInfo.delta,fou=False,lnorm=1.)
        gausKer2=gausKer2**2.
        gausKer2=np.fft.fft2(gausKer2)
        
        # smothing with Gausian Kernel
        g1OutTmp=np.fft.ifft2(np.fft.fft2(g1OutTmp)*gausKer).real
        g2OutTmp=np.fft.ifft2(np.fft.fft2(g2OutTmp)*gausKer).real
        weightOut=np.fft.ifft2(np.fft.fft2(weightOutTmp)*gausKer).real
        sigma2Out=np.fft.ifft2(np.fft.fft2(sigma2)*gausKer2).real
        gausKerCon=np.fft.ifft2(np.fft.fft2(mask0.astype(float))*gausKer).real
    
    g1OutTmp/=aveWeight
    g2OutTmp/=aveWeight
    weightOut/=aveWeight
    mask=weightOut>0.5
    sigma2Out[~mask]=0.
    sigma2Out[mask]=sigma2Out[mask]/gausKerCon[mask]**2.
    g1Out=np.zeros(gridInfo.shape)
    g1Out[mask]=g1OutTmp[mask]/weightOut[mask]
    g2Out=np.zeros(gridInfo.shape)
    g2Out[mask]=g2OutTmp[mask]/weightOut[mask]
    pyfits.writeto(os.path.join(outDir,'%s_dnnz_10_bins_g1Map.fits' %fieldname),g1Out,overwrite=True)
    pyfits.writeto(os.path.join(outDir,'%s_dnnz_10_bins_g2Map.fits' %fieldname),g2Out,overwrite=True)
    pyfits.writeto(os.path.join(outDir,'%s_dnnz_10_bins_weightMap.fits' %fieldname),weightOut,overwrite=True)
    pyfits.writeto(os.path.join(outDir,'%s_dnnz_10_bins_std.fits' %fieldname),np.sqrt(sigma2Out),overwrite=True)
    
    varOut=np.zeros(gridInfo.shape,dtype=float)
    del g1Out,g2Out,weightOut
    gc.collect()

1565.6270882643867 -563.4529855801278


/home/xiangchong/work/ana/lsstsw/miniconda/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/ipykernel_launcher.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "overwrite" which is no longer supported as of 3.3 and will become an error two minor releases later


7511.884040200001 -293.94776607479395


/home/xiangchong/work/ana/lsstsw/miniconda/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/ipykernel_launcher.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "overwrite" which is no longer supported as of 3.3 and will become an error two minor releases later


9056.585232079698 -280.5903829327962


...

/home/xiangchong/work/ana/lsstsw/miniconda/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/ipykernel_launcher.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "overwrite" which is no longer supported as of 3.3 and will become an error two minor releases later


19648.172504524035 -240.3134553150722


/home/xiangchong/work/ana/lsstsw/miniconda/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/ipykernel_launcher.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "overwrite" which is no longer supported as of 3.3 and will become an error two minor releases later


12574.431102030565 2352.7539675342996


/home/xiangchong/work/ana/lsstsw/miniconda/envs/lsst-scipipe-cb4e2dc/lib/python3.7/site-packages/ipykernel_launcher.py:25: MatplotlibDeprecationWarning: savefig() got unexpected keyword argument "overwrite" which is no longer supported as of 3.3 and will become an error two minor releases later
